In [3]:
with open("./api_key.txt", "r") as f:
    api_key = f.readline()
    
with open("./tavily_key.txt", "r") as f:
    tavily_key = f.readline()

In [4]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langgraph.checkpoint.sqlite import SqliteSaver

In [5]:
import os

os.environ["TAVILY_API_KEY"] = tavily_key

In [6]:
tool = TavilySearchResults(max_results=2)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [7]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the messages state key
with the default operator.add or + reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
messages key with a customer reducer function, which replaces
messages with the same id, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [10]:
memory = SqliteSaver.from_conn_string(":memory:")

In [11]:
class Agent:

    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_chatgroq)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=['action'])
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_chatgroq(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [12]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatGroq(
    model="llama3-8b-8192",  # Ensure this model ID is correct
    temperature=0.7,
    api_key=api_key
)
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

State Management: The graph maintains its state across executions. By passing None as input, you're instructing the graph to continue from its last known state.


Thread Configuration: Ensuring the thread_id remains consistent allows the graph to track and manage the execution context appropriately.

In [22]:
from langgraph.checkpoint.sqlite import SqliteSaver

with SqliteSaver.from_conn_string(":memory:") as memory:
    abot = Agent(model, [tool], system=prompt, checkpointer=memory)
    messages = [HumanMessage(content='What is the weather in Patna?')]
    thread = {"configurable": {"thread_id": "1"}}
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v)
    print(abot.graph.get_state(thread))
    print(abot.graph.get_state(thread).next)
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)
    print(abot.graph.get_state(thread))
    print(abot.graph.get_state(thread).next)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_x2sm', 'function': {'arguments': '{"query":"weather in Patna"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 2052, 'total_tokens': 2125, 'completion_time': 0.060833333, 'prompt_time': 0.473400555, 'queue_time': -0.622239593, 'total_time': 0.534233888}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6825d3b9-0e63-432a-9743-5969316827d4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Patna'}, 'id': 'call_x2sm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2052, 'output_tokens': 73, 'total_tokens': 2125})]}
()
StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in Patna?', additional_kwargs={}, response_metadata={}, id='cd7f5648-30d0-44c7-9dd1-73b3501d3189'), AIM

In [25]:
# Initialize SqliteSaver with the file path
with SqliteSaver.from_conn_string(":memory:") as memory:
    # Initialize your agent with the SQLite-based memory
    abot = Agent(model, [tool], system=prompt, checkpointer=memory)

    # Define initial messages and thread configuration
    messages = [HumanMessage(content="Who is the Rahul Kumar working in Bosch as senior software engineer?")]
    thread = {"configurable": {"thread_id": "2"}}

    # Start processing messages
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v)

    # Loop to continue processing based on the current state
    while abot.graph.get_state(thread).next:
        print("\nCurrent State:", abot.graph.get_state(thread), "\n")
        
        # User input to control the next step
        _input = input("Proceed? (y to continue, any other key to abort): ")
        if _input.lower() != "y":
            print("Aborting process.")
            break

        # Continue execution without new messages
        for event in abot.graph.stream(None, thread):
            for v in event.values():
                print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mxyy', 'function': {'arguments': '{"query":"Rahul Kumar Bosch senior software engineer"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 1020, 'total_tokens': 1104, 'completion_time': 0.07, 'prompt_time': 0.118527458, 'queue_time': 0.007519612000000009, 'total_time': 0.188527458}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a91ccbb6-9257-49d2-88c5-843bafecabc7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Rahul Kumar Bosch senior software engineer'}, 'id': 'call_mxyy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1020, 'output_tokens': 84, 'total_tokens': 1104})]}
()

Current State: StateSnapshot(values={'messages': [HumanMessage(content='Who is the Rahul Kumar working in Bosch as senior soft

Handling agent state by updating / modifying the query in between

In [45]:
with SqliteSaver.from_conn_string(":memory:") as memory:
    abot = Agent(model, [tool], system=prompt, checkpointer=memory)
    messages = [HumanMessage(content='What is the weather in Patna?')]
    thread = {"configurable": {"thread_id": "3"}}
    
    # Start initial processing
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v)
    
    # Display the current state and the next action
    print("Initial state:", abot.graph.get_state(thread))
    print("Next action:", abot.graph.get_state(thread).next)
    
    # Access and print the current tool calls for verification
    current_values = abot.graph.get_state(thread)
    print("Current tool calls:", current_values.values['messages'][-1].tool_calls)
    
    # Update tool calls to modify the query
    print("Updating search query to Bengaluru.")
    existing_tool_call = current_values.values['messages'][-1].tool_calls[0]
    existing_tool_call['args']['query'] = 'current weather in Bengaluru'
    
    # Apply the update to the graph’s state
    abot.graph.update_state(thread, current_values.values)
    
    # Confirm the updated tool call
    updated_values = abot.graph.get_state(thread)
    print("Updated tool calls:", updated_values.values['messages'][-1].tool_calls)
    
    # Continue processing from the modified state
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

    # Print the final state after all processing
    print("Final state:", abot.graph.get_state(thread))
    print("Next action:", abot.graph.get_state(thread).next)
    
        # Continue processing from the modified state
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

    # Print the final state after all processing
    print("Final state:", abot.graph.get_state(thread))
    print("Next action:", abot.graph.get_state(thread).next)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ma0c', 'function': {'arguments': '{"query":"weather in Patna"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 2051, 'total_tokens': 2131, 'completion_time': 0.066666667, 'prompt_time': 0.560617483, 'queue_time': -0.6479681820000001, 'total_time': 0.62728415}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-99e359b3-e3d5-4892-ade4-a52aba2cd4d0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Patna'}, 'id': 'call_ma0c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2051, 'output_tokens': 80, 'total_tokens': 2131})]}
()
Initial state: StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in Patna?', additional_kwargs={}, response_metadata={}, id='c4223783-0d9e-43dd-84e